# N09. Algortimos Genéticos y Función de Ackley

__Borja González Seoane, Computación Inteligente y Ética de la IA. Curso 2022-23__

## Preámbulo

Este nuevo _notebook_ sobre algoritmos genéticos es una copia de N08. En este caso, se parte de la implementación alcanzada en el anterior _notebook_ y mejora el algoritmo genético dotándolo de más flexibilidad en cuanto a parametrización. También se añaden nuevas piezas necesarias para poder abarcar el problema de optimización de la función de Ackley.


In [1]:
# %pip install numpy

In [2]:
from typing import Callable, Tuple
import numpy as np
import random

## Función de Ackley

Se propone un problema de optimización típico, como lo es la optimización de una función continua. La función de Ackley es una función de dos variables que tiene un mínimo global teórico en (0, 0), de valor 0. La función de Ackley se define como:

$$
f(x, y) = -20 \exp\left(-0.2 \sqrt{0.5(x^2 + y^2)}\right) - \exp\left(0.5(\cos(2\pi x) + \cos(2\pi y))\right) + 20 + e
$$

Para más información, se puede consultar https://www.sfu.ca/~ssurjano/ackley.html.

Para con el propósito de este _notebook_, lo más relevante es que aunque la función de Ackley tiene un mínimo global en (0, 0), de valor 0, también tiene muchos mínimos locales. Un algoritmo de optimización puede tener dificultades para encontrar el mínimo global si se estanca en un mínimo local. Así pues, constituye un problema de optimización no trivial interesante para probar la implementación del algoritmo genético.

La implementación del algoritmo genético deberá ser capaz de:

- Representar individuos con valores reales.
- Minimizar en lugar de maximizar.

Así pues, son necesarias algunas modificaciones en la implementación del algoritmo genético de N08.

No obstante, lo primero es implementar la función de Ackley.

In [3]:
def fun_ackley(individuo: np.ndarray) -> float:
    """
    Función de Ackley para probar el algoritmo genético.

    :param individuo: Individuo a evaluar.
    :return: Valor de la función de Ackley para el individuo.
    """
    # Se comprueba que el individuo sea de dimensión 2
    assert len(individuo) == 2, "El individuo debe ser de dimensión 2."

    a = 20
    b = 0.2
    c = 2 * np.pi

    x = individuo[0]
    y = individuo[1]

    termino_1 = -a * np.exp(-b * np.sqrt(0.5 * (x**2 + y**2))) # Primera parte función ackley
    termino_2 = -np.exp(0.5 * (np.cos(c * x) + np.cos(c * y))) # Segunda parte función ackley

    return termino_1 + termino_2 + a + np.exp(1) # Función de ackley entera


# Pruebas
test = fun_ackley(np.random.rand(2))
display(test)

# Se comprueba el mínimo teórico de la función de Ackley. Hay que tener en cuenta la precisión
# del tipo de dato flotante en Python, que es del orden de 10^-16. Ergo se considera error si la
# desviación es mayor a 10^-15
test = fun_ackley(np.array([0, 0]))
display(test)
assert np.isclose( # assert es para verificar si la condición de que está cerca es verdadera
    test, 0, atol=1e-15
), f"Función mal implementada. Resultado diferente al esperado."

3.9677748886824635

4.440892098500626e-16

In [4]:
def fun_ackley_neg(individuo: np.ndarray) -> float:
    return -fun_ackley(individuo)

## Recuperar piezas necesarias de N08


## Implementar las nuevas piezas necesarias para el algoritmo

In [5]:
# Para pruebas

individuo_test = np.array([1, 0, 1, 1, 0, 1, 0, 0, 1, 1])

display(individuo_test)

poblacion_test = np.array(
    [
        individuo_test,
        [0, 1, 1, 0, 1, 0, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    ]
)

display(poblacion_test)

array([1, 0, 1, 1, 0, 1, 0, 0, 1, 1])

array([[1, 0, 1, 1, 0, 1, 0, 0, 1, 1],
       [0, 1, 1, 0, 1, 0, 1, 1, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

### Modificaciones necesarias del algoritmo genético

Para adaptar el algoritmo genético al problema de optimización de la función de Ackley, se requieren las siguientes modificaciones:

- Representar individuos con valores reales.
- Modificar la función de evaluación para que calcule el valor de la función de Ackley.

Así pues, se añaden dos piezas adicionales al algoritmo genético y se modifica la firma del algortimo genético para que se puedan pasar todas las piezas necesarias como argumentos, permitiendo así una mayor flexibilidad. También se añaden algunas mejoras adicionales, como la consideración del elitismo.

In [6]:
def mutacion(individuo: np.ndarray, prob_mutacion: float, representacion_reales: bool) -> np.ndarray:
    if representacion_reales:
        # Generar una máscara de mutación que tiene la misma longitud del individuo pero lleno de números boolenaos.
        # Para cada elemento del individuo se genera un número aleatorio entre 0 y 1, si es menor que la probabilidad de mutación el valor será True y se mutará.
        mascara_mutacion = np.random.uniform(size=individuo.shape) < prob_mutacion

        # Añadir un pequeño valor aleatorio a los elementos mutados, de media 0 y desviación típica 0.1
        individuo[mascara_mutacion] += np.random.normal(scale=0.1, size=np.sum(mascara_mutacion))
    else:
        for i in range(len(individuo)):
            if random.random() < prob_mutacion:
                individuo[i] = 1 - individuo[i]  # Cambia 0 a 1 y viceversa
    return individuo

# Seleccion mediante el método de torneo
def seleccion(poblacion: np.ndarray, fun_eval, tam_torneo: int) -> np.ndarray:
    # Selecciona 'tam_torneo' individuos al azar de la población
    competidores = random.choices(poblacion, k=tam_torneo)
    
    # Evalúa la aptitud de los competidores
    aptitudes = np.apply_along_axis(fun_eval, 1, competidores)
    
    # Encuentra el índice del competidor con la mayor aptitud
    ganador_idx = np.argmax(aptitudes)
    
    # Devuelve el ganador
    return competidores[ganador_idx]

def reproduccion(individuo1: np.ndarray, individuo2: np.ndarray, representacion_reales: bool) -> Tuple[np.ndarray, np.ndarray]:
    punto_cruce = random.randint(1, len(individuo1)-1)
    hijo1 = np.concatenate((individuo1[:punto_cruce], individuo2[punto_cruce:]))
    hijo2 = np.concatenate((individuo2[:punto_cruce], individuo1[punto_cruce:]))
    return hijo1, hijo2

In [7]:
def algoritmo_genetico(
    fun_eval: Callable[[np.ndarray], float], # Función de evaluación (fun_ackley)
    tam_cromosoma: int,
    tam_poblacion: int,
    num_generaciones: int,
    tam_torneo: int,
    prob_mutacion: float = 0.01,
    representacion_reales: bool = False, # hiperparámetro para representar los individuos como vector de dos números binarios o números reales
) -> Tuple[np.ndarray, float]:
    # Inicializar población
    if representacion_reales:
        # En el caso de que se representen los individuos con números reales, generamos una población de individuos con valores aleatorios entre -10 y 10
        poblacion = np.random.uniform(low=-10.0, high=10.0, size=(tam_poblacion, tam_cromosoma))
    else:
        poblacion = np.random.randint(2, size=(tam_poblacion, tam_cromosoma))

    for _ in range(num_generaciones):
        # Evaluar población
        aptitudes = np.apply_along_axis(fun_eval, 1, poblacion)

        # Seleccionar padres
        padre1 = seleccion(poblacion, fun_eval, tam_torneo)
        padre2 = seleccion(poblacion, fun_eval, tam_torneo)

        # Reproducción
        hijo1, hijo2 = reproduccion(padre1, padre2, representacion_reales)

        # Mutación
        hijo1 = mutacion(hijo1, prob_mutacion, representacion_reales)
        hijo2 = mutacion(hijo2, prob_mutacion, representacion_reales)

        # Reemplazar dos individuos menos aptos
        indices_menos_aptos = np.argpartition(aptitudes, 2)[:2]
        poblacion[indices_menos_aptos[0]] = hijo1
        poblacion[indices_menos_aptos[1]] = hijo2

    # Evaluar población final
    aptitudes = np.apply_along_axis(fun_eval, 1, poblacion)

    # Encontrar el mejor individuo
    indice_mejor = np.argmax(aptitudes)
    mejor_individuo = poblacion[indice_mejor]
    mejor_aptitud = aptitudes[indice_mejor]

    return mejor_individuo, mejor_aptitud

In [9]:
# Parámetros del algoritmo genético
tam_cromosoma = 2
tam_poblacion = 500
num_generaciones = 1000
prob_mutacion = 0.01
tam_torneo = 200
representacion_reales = True

# Ejecutar el algoritmo genético
mejor_individuo, mejor_aptitud = algoritmo_genetico(
    fun_ackley_neg,
    tam_cromosoma,
    tam_poblacion,
    num_generaciones,
    tam_torneo,
    prob_mutacion,
    representacion_reales
)

print("Mejor individuo: ", mejor_individuo)
print("Mejor aptitud: ", mejor_aptitud)

Mejor individuo:  [0 0]
Mejor aptitud:  -4.440892098500626e-16
